In [1]:
import os, sys
# Use the notebook’s current directory as the base
notebook_dir = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_dir, '..')))

# Import all functions from model_fitting.util
from util import *

In [2]:
import numpy as np
import pytensor.tensor as pt
import pymc as pm
from pymc.math import logdiffexp
from pymc.distributions.dist_math import normal_lcdf, log_normal, check_parameters

# Test a Simple Function

## Gauss-Legendre Quadrature

In [1]:


def gauss_legendre_quadrature(f, a, b, n):
    """Gauss-Legendre quadrature integration."""
    [x, w] = np.polynomial.legendre.leggauss(n)  # n-point Gauss-Legendre
    # Transform x from [-1, 1] to [a, b]
    t = 0.5 * (x + 1) * (b - a) + a
    integral = 0.5 * (b - a) * np.sum(w * f(t))
    return integral

# Example function to integrate
def integrand(x):
    return 1.0 / (x**2 + 1.005)

# Compare with GSL integration
a, b = 0, 1  # Integration limits
n = 41  # Number of Gauss-Legendre points

result_gauss_legendre = gauss_legendre_quadrature(integrand, a, b, n)
print(f"Gauss-Legendre Quadrature Result: {result_gauss_legendre}")

Gauss-Legendre Quadrature Result: 0.7821982220345253


# Test Successful Inhibition Likelihood

In [3]:
# Sample input parameters
mu_go, sigma_go, tau_go = 1000, 50, 100  # Go process
mu_stop, sigma_stop, tau_stop = 1000, 30, 80  # Stop process
p_tf = 0.5  # Probability of triggering stop
# ssd = np.array([200, 250, 300])  # Stop-signal delays
ssd = np.array([0]) # Stop-signal delays
t_r = np.array([350, 400, 450])  # Response times
n_points = 1000  # Number of quadrature points

# Upper bound for integration (could be RT ceiling)
upper_bound = np.array([10000])

# Precompute nodes and weights for Legendre quadrature
nodes, weights = precompute_legendre_quadrature(ssd, upper_bound, n_points)

In [4]:
integral_result = integrate_cexgauss(nodes, weights, 
                                     mu_go, sigma_go, tau_go,
                                     mu_stop, sigma_stop, tau_stop,
                                     ssd)
print(f"Integral Result: {integral_result.eval()}")

Integral Result: [0.54923889]


In [7]:
# Calculate log-likelihood of go trials 
logpdf_exgaussian(t_r, mu_go, sigma_go, tau_go)
go_log_likelihood = logpdf_exgaussian(t_r, mu_go, sigma_go, tau_go)
print(f"Go Log-Likelihood: {np.sum(go_log_likelihood.eval())}")

Go Log-Likelihood: -241.16222968163703


In [8]:
# Calculate log-likelihood of stop respond trials 
stop_log_likelihood = stop_respond_log_likelihood(t_r, mu_go, sigma_go, tau_go,
                                             mu_stop, sigma_stop, tau_stop,
                                             p_tf, ssd)
print(f"Stop Log-Likelihood: {np.sum(stop_log_likelihood.eval())}")

Stop Log-Likelihood: -241.16222968163703
